In [29]:
import pandas as pd
import time
import numpy as np
from tqdm import tqdm
from tqdm.auto import tqdm
import astropy
from astropy import units as u
from astropy.coordinates import SkyCoord


path = '/home/masha/AstroAugumetation/ANTARES/'
path_to = "/home/masha/AstroAugumetation/ZTF_BTS_02_02__02_04_2021.csv"


diff_between_format_bts_and_mjd = 2458000 - 2400000.5
calculate_mjd = lambda x: x + diff_between_format_bts_and_mjd
calculate_flux = lambda x: 10**((x + 48.6) / -2.5)


bts = pd.read_csv(path_to, sep =',')
bts = bts.drop('Unnamed: 0', 1) 


data = []
new_bts_id = []
for ztf_id in tqdm(bts['ZTFID'].values):
    try:
        df = pd.read_csv(path + ztf_id + '.csv')
        if df.index.values.shape[0] != 0:
            df = df.drop('Unnamed: 0', 1)
            mask_g = df[df['ant_passband'] == 'g'].index.shape[0] > 10
            mask_r = df[df['ant_passband'] == 'R'].index.shape[0] > 10

            if (mask_g)&(mask_r):
                df = df[df['ant_passband'] != 'i']
                #attention, умножаю на 100 просто для хороших значений для обучения 
                df['ant_mag'] = 100*1e26*df['ant_mag'].map(calculate_flux) #* 1e26 # переводим в потоки в [мЯн]
                df['ant_magerr'] = 1.086*df['ant_magerr']*df['ant_mag']
                df['ant_passband'][df['ant_passband'] == 'g'] = 0
                df['ant_passband'][df['ant_passband'] == 'R'] = 1
                df.rename(columns={"ant_mjd": "mjd","ant_mag": "flux", \
                                   "ant_magerr": "flux_err", "ant_passband": "passband"},
                          inplace=True)
                mjd = df['mjd'].values
                mask_gaps = np.diff(mjd, 1).max() < 1000
                if mask_gaps:
                    df.insert(0, 'object_id', ztf_id)
                    target = bts[bts['ZTFID'] == ztf_id]['type'].values[0]
                    df.insert(5, 'obj_type', target)
                    peakt = bts[bts['ZTFID'] == ztf_id]['peakt'].map(calculate_mjd).values[0]
                    df.insert(6, 'peak_time', peakt)
                    data.append(df)
                    new_bts_id.append(ztf_id)
    except FileNotFoundError:
        continue

print(len(data), len(new_bts_id))

  0%|          | 0/2736 [00:00<?, ?it/s]

<ipython-input-29-32aa773d680d>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ant_passband'][df['ant_passband'] == 'g'] = 0
<ipython-input-29-32aa773d680d>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ant_passband'][df['ant_passband'] == 'R'] = 1


1403 1403


In [30]:
data[0]

,object_id,mjd,passband,flux,flux_err,obj_type,peak_time
0,ZTF18aajpjdi,58242.252847,1,14.157950,0.425610,SN Ia,58245.21
1,ZTF18aajpjdi,58245.207500,1,14.925189,2.228412,SN Ia,58245.21
2,ZTF18aajpjdi,58248.252627,1,10.866266,0.507728,SN Ia,58245.21
3,ZTF18aajpjdi,58248.254537,1,12.107097,0.443637,SN Ia,58245.21
4,ZTF18aajpjdi,58255.232431,1,9.815668,0.664949,SN Ia,58245.21
...,...,...,...,...,...,...,...
204,ZTF18aajpjdi,59290.437859,0,8.961732,0.944889,SN Ia,58245.21
205,ZTF18aajpjdi,59291.303449,0,3.432468,0.686505,SN Ia,58245.21
206,ZTF18aajpjdi,59297.304745,0,7.066426,2.060123,SN Ia,58245.21
207,ZTF18aajpjdi,59303.359861,0,9.349622,1.921683,SN Ia,58245.21


In [31]:
data[0].dropna() 

,object_id,mjd,passband,flux,flux_err,obj_type,peak_time
0,ZTF18aajpjdi,58242.252847,1,14.157950,0.425610,SN Ia,58245.21
1,ZTF18aajpjdi,58245.207500,1,14.925189,2.228412,SN Ia,58245.21
2,ZTF18aajpjdi,58248.252627,1,10.866266,0.507728,SN Ia,58245.21
3,ZTF18aajpjdi,58248.254537,1,12.107097,0.443637,SN Ia,58245.21
4,ZTF18aajpjdi,58255.232431,1,9.815668,0.664949,SN Ia,58245.21
...,...,...,...,...,...,...,...
204,ZTF18aajpjdi,59290.437859,0,8.961732,0.944889,SN Ia,58245.21
205,ZTF18aajpjdi,59291.303449,0,3.432468,0.686505,SN Ia,58245.21
206,ZTF18aajpjdi,59297.304745,0,7.066426,2.060123,SN Ia,58245.21
207,ZTF18aajpjdi,59303.359861,0,9.349622,1.921683,SN Ia,58245.21


In [23]:
df_all = pd.concat(data, axis=0)

In [24]:
df_all

,object_id,mjd,passband,flux,flux_err,obj_type,peak_time
0,ZTF18aajpjdi,58242.252847,1,14.157950,0.425610,SN Ia,58245.21
1,ZTF18aajpjdi,58245.207500,1,14.925189,2.228412,SN Ia,58245.21
2,ZTF18aajpjdi,58248.252627,1,10.866266,0.507728,SN Ia,58245.21
3,ZTF18aajpjdi,58248.254537,1,12.107097,0.443637,SN Ia,58245.21
4,ZTF18aajpjdi,58255.232431,1,9.815668,0.664949,SN Ia,58245.21
...,...,...,...,...,...,...,...
19,ZTF21aaobcrz,59301.490151,0,11.537097,1.767728,SN Ia,59291.39
20,ZTF21aaobcrz,59303.383657,0,9.336705,1.916483,SN Ia,59291.39
21,ZTF21aaobcrz,59303.410012,0,7.931224,1.395618,SN Ia,59291.39
22,ZTF21aaobcrz,59303.430451,1,12.169447,1.749869,SN Ia,59291.39


In [25]:
df_all.to_csv('ANTARES.csv')